In [3]:
import pandas as pd
import numpy as np

import re

import sys
#Görselleştirme
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import warnings
# warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

In [4]:
first_data_path ="../Web Scraping/home_data.xlsx"
last_data_path="preprocessing_data.xlsx"

df=pd.read_excel(first_data_path)  #preprocessing yapılmamış verilerin okunması


In [5]:
#bazı kelime ve noktalama işaretleriden arındırma
df = df.drop_duplicates()
df['Ev Türü'] = df['Ev Türü'].str.replace('home', '', regex=True)
df['Oda Sayısı'] = df['Oda Sayısı'].str.replace('weekend', '', regex=True)
df['Kat'] = df['Kat'].str.replace('layers', '', regex=True)
df['Metrekare'] = df['Metrekare'].str.replace('texture', '', regex=True)
df['Metrekare'] = df['Metrekare'].str.replace('m2', '', regex=True)
df['Fiyat'] = df['Fiyat'].str.replace('TL', '', regex=True)
df['Fiyat'] = df['Fiyat'].str.replace('.', '', regex=True)
df['Fiyat'] = df['Fiyat'].str.replace('GBP', '', regex=True)

df['Konum'] = df['Konum'].apply(lambda x: x.split('-')[0].strip()) #- iconunu gördüğünde alt satıra alıp orayı kaldır

# '+' işaretinden ayırıp, sayıları topla ve yeni sütun oluştur
def calculate_room_count(text):
    try:
        return sum(map(int, text.split('+')))
    except ValueError:
        return np.nan  # Hata durumunda NaN (null) değeri döndür

df['Oda Sayısı'] = df['Oda Sayısı'].apply(calculate_room_count)

df.drop(columns=['Başlık'], inplace=True)
df.drop(columns=['Ev Türü'], inplace=True)

In [7]:
#dosyadaki yabancı karakterleri temizle
def temizle(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                              u"\U0001F300-\U0001F5FF"  # Simge ve semboller
                               u"\U0001F680-\U0001F6FF"  # Transport ve simgeler
                               u"\U0001F1E0-\U0001F1FF"  # Bayraklar (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Verilerdeki emoji karakterlerini temizleme
df['Oda Sayısı'] = df['Oda Sayısı'].apply(lambda x: temizle(x) if isinstance(x, str) else x)
df['Kat'] = df['Kat'].apply(lambda x: temizle(x) if isinstance(x, str) else x)
df['Metrekare'] = df['Metrekare'].apply(lambda x: temizle(x) if isinstance(x, str) else x)
df['Fiyat'] = df['Fiyat'].apply(lambda x: temizle(x) if isinstance(x, str) else x)


In [16]:
df.dropna(axis=0, inplace=True)

In [17]:
df.to_excel(last_data_path, index=False)

In [18]:
df=pd.read_excel("preprocessing_data.xlsx")

In [19]:
#encode etme işlemleri
label_encoder=LabelEncoder() #labelencoder objesi oluşturuldu
df["Konum"]=label_encoder.fit_transform(df["Konum"])  #konum sütunu encode edildi
df["Kat"]=label_encoder.fit_transform(df["Kat"])  #kat sütunu encode edildi

result_df=df[['Oda Sayısı','Kat','Metrekare','Fiyat','Konum']]

result_df.to_excel("encoded_data.xlsx", index= False)

result_df = result_df.drop_duplicates()
result_df.to_excel("encoded_data.xlsx", index= False)

In [21]:
result_df.head()

,Oda Sayısı,Kat,Metrekare,Fiyat,Konum
0,5,6,220,3750000,13
1,4,9,145,4450000,14
2,4,9,231,7500000,1
3,5,9,145,4100000,14
4,6,9,200,8750000,2


In [23]:
result_df.info()  # Veri İle İlgili kısa bilgiler : veri türü, boş alan var yok 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Oda Sayısı  57 non-null     int64
 1   Kat         57 non-null     int32
 2   Metrekare   57 non-null     int64
 3   Fiyat       57 non-null     int64
 4   Konum       57 non-null     int32
dtypes: int32(2), int64(3)
memory usage: 2.2 KB


In [24]:
# Gerekli Kütüphaneleri Yükleyelim

from sklearn.model_selection import learning_curve, RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [10]:
from sklearn.model_selection import train_test_split
# Öncelikle, bağımsız değişkenlerin (X) ve bağımlı değişkenin (y) seçilmesi
x = result_df[['Oda Sayısı', 'Kat', 'Metrekare', 'Konum']]  # Bağımsız değişkenler
y = result_df['Fiyat']  # Bağımlı değişken

# Train-test split işlemi
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [11]:
# Modelleri Denemek İçin Çağılarım (LinearRegression)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import joblib

# Lineer Regresyon Train
regressor = LinearRegression()
regressor.fit(x_train,y_train)
# 
y_predict = regressor.predict(x_test) # üretilen modeli (lr) test vverisi ile deneyelim


ML_RSCORE = r2_score(y_test, y_predict) 

print(ML_RSCORE)  #R^2 değerleri
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_predict)))

0.29666455976954187
 root mean squared error:  2620662.8103589993


In [12]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

# Random Forest Regressor modelini oluşturma ve eğitme
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
regressor.fit(x_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred = regressor.predict(x_test)

# R2 skorunu hesaplama
ML_RSCORE = r2_score(y_test, y_pred)

print(ML_RSCORE)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_pred)))

joblib.dump(regressor, "randomForest_model.pkl")

0.3204209949812499
 root mean squared error:  2576023.8023102265


['randomForest_model.pkl']

In [13]:
# Lasso Regresyon
from sklearn.linear_model import Lasso

# Lasso Regresyon modelini oluşturma ve eğitme
lasso_regressor = Lasso(alpha=0.1, random_state=10)  # alpha değeri değiştirilebilir
lasso_regressor.fit(x_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred_lasso = lasso_regressor.predict(x_test)

# R2 skorunu hesaplama
ML_RSCORE_LASSO = r2_score(y_test, y_pred_lasso)

print(ML_RSCORE_LASSO)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_pred_lasso)))

0.296664562363528
 root mean squared error:  2620662.8055263385


In [14]:
# Ridge Regresyon
from sklearn.linear_model import Ridge

# Ridge Regresyon modelini oluşturma ve eğitme
ridge_regressor = Ridge(alpha=0.5, random_state=0)  # alpha değeri değiştirilebilir
ridge_regressor.fit(x_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred_ridge = ridge_regressor.predict(x_test)

# R2 skorunu hesaplama
ML_RSCORE_RIDGE = r2_score(y_test, y_pred_ridge)

print(ML_RSCORE_RIDGE)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_pred_ridge)))

0.2966812398929951
 root mean squared error:  2620631.7346902075


In [15]:
#Polynomial Features Regressor
from sklearn.preprocessing import PolynomialFeatures

# Polinominal özelliklerin oluşturulması
poly = PolynomialFeatures(degree=2)  # İstediğiniz derecede polinom oluşturabilirsiniz
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)

# Polinominal regresyon modelinin eğitilmesi
poly_regressor = LinearRegression()
poly_regressor.fit(x_train_poly, y_train)

# Test seti üzerinde tahmin yapma
y_pred_poly = poly_regressor.predict(x_test_poly)

# R2 skorunu hesaplama
ML_RSCORE_POLY = r2_score(y_test, y_pred_poly)
print(ML_RSCORE_POLY)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_pred_poly)))

0.34608720606724874
 root mean squared error:  2526910.2216077177


In [16]:
#Elastic Net Regressor
from sklearn.linear_model import ElasticNet

poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)

scaler = StandardScaler()
x_train_poly_scaled = scaler.fit_transform(x_train_poly)
x_test_poly_scaled = scaler.transform(x_test_poly)

elastic_net_regressor = ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000, tol=0.001, random_state=0)
elastic_net_regressor.fit(x_train_poly_scaled, y_train)

y_pred_elastic_net = elastic_net_regressor.predict(x_test_poly_scaled)

ML_RSCORE_ELASTICNET_POLY = r2_score(y_test, y_pred_elastic_net)
print(ML_RSCORE_ELASTICNET_POLY)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_pred_elastic_net)))

0.28934291208400953
 root mean squared error:  2634267.9067271897


In [17]:
#Gradient Boosting Regressor 
from sklearn.ensemble import GradientBoostingRegressor

# Polinom özelliklerin oluşturulması
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)

# Verilerin ölçeklendirilmesi
scaler = StandardScaler()
x_train_poly_scaled = scaler.fit_transform(x_train_poly)
x_test_poly_scaled = scaler.transform(x_test_poly)

# Gradient Boosting Regresyon modelinin oluşturulması ve eğitilmesi
gradient_boosting_regressor = GradientBoostingRegressor(n_estimators=100, random_state=0)
gradient_boosting_regressor.fit(x_train_poly_scaled, y_train)

# Test seti üzerinde tahmin yapma
y_pred_gradient_boosting = gradient_boosting_regressor.predict(x_test_poly_scaled)

# R2 skorunu hesaplama
ML_RSCORE_GRADIENT_BOOSTING = r2_score(y_test, y_pred_gradient_boosting)
print(ML_RSCORE_GRADIENT_BOOSTING)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_pred_gradient_boosting)))

0.5084721617902801
 root mean squared error:  2190806.1667748233


In [18]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(criterion='squared_error',splitter='best')
dt.fit(x_train,y_train)
y_predict_dt = dt.predict(x_test)

ML_RSCORE_DECISION_TREE = r2_score(y_test, y_predict_dt)
print(ML_RSCORE_DECISION_TREE)
print(' root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_predict_dt)))

-0.30232928238612233
 root mean squared error:  3566073.562431286


In [19]:
final_RMSE = pd.DataFrame( data  = [[4212497.69, 3268657.23, 4212497.69,  4212490.09,   3967573.65,  4231376.26, 3337029.16, 3656493.75],
                                    [0.136,0.479,0.136,0.136,0.233,0.128,0.457,0.349]],
                          columns  = ['Linear Regression','Random Forest Regressor','Lasso','Ridge','Polynomial Features','Elastic Net','Gradient Boosting','Decision Tree'],
                         index =['RMSE','R2'])


final_RMSE = final_RMSE.T

cm = sns.light_palette('red', as_cmap=True)

final_RMSE = final_RMSE.style.background_gradient(cmap=cm)
final_RMSE

,RMSE,R2
Linear Regression,4212497.690000,0.136000
Random Forest Regressor,3268657.230000,0.479000
Lasso,4212497.690000,0.136000
Ridge,4212490.090000,0.136000
Polynomial Features,3967573.650000,0.233000
Elastic Net,4231376.260000,0.128000
Gradient Boosting,3337029.160000,0.457000
Decision Tree,3656493.750000,0.349000


In [20]:
from sklearn.model_selection import GridSearchCV
# tüm parametreler için değil sadece iki temel parametre için arama yapıyoruz!!!
params_grid = {
    'n_estimators':[10,20,50],
    'random_state':list(range(0,5))}


grid_search = GridSearchCV(RandomForestRegressor(min_samples_split=2,bootstrap=False,random_state=42), params_grid, verbose=1, cv=3)

grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


GridSearchCV(cv=3,
             estimator=RandomForestRegressor(bootstrap=False, random_state=42),
             param_grid={'n_estimators': [10, 20, 50],
                         'random_state': [0, 1, 2, 3, 4]},
             verbose=1)

In [43]:
grid_search.best_params_    # en uygun parametreleri yazdır

{'n_estimators': 50, 'random_state': 1}

In [21]:
# bulduğumuz parametrelerle test edelim   #yeni parametreler denedim fakat daha iyi sonuç vermedi ilk parametrelerle devam ettim
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10,random_state=0)
rf.fit(x_train,y_train)
y_predict_rf = rf.predict(x_test)

print('ML_RSCORE: ',r2_score(y_test,y_predict_rf))
print('root mean squared error: ',np.sqrt(mean_squared_error(y_test,y_predict_rf)))

ML_RSCORE:  0.2830996038488832
root mean squared error:  2645813.9693755414
